In [ ]:
device = "cpu"

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset, random_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
zip_path= '/content/drive/My Drive/ISPP_Folder/CIFAKE.zip'

import zipfile
import os

# Define the destination directory for unzipped files
extract_to = '/content/unzipped_folder/'

# Ensure the destination directory exists
os.makedirs(extract_to, exist_ok=True)

# Open and extract the ZIP file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("Files unzipped successfully.")

Files unzipped successfully.


In [ ]:
#load train datasets

shadow_dataset0 = torch.load('custom_dataset_0.pth')
shadow_dataset1 = torch.load('custom_dataset_1.pth')
shadow_dataset2 = torch.load('custom_dataset_2.pth')
shadow_dataset3 = torch.load('custom_dataset_3.pth')
shadow_dataset4 = torch.load('custom_dataset_4.pth')

<ipython-input-22-30112dfc2448>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  shadow_dataset0 = torch.load('custom_dataset_0.pth')
<ipython-input-22-30112dfc2448>:4: Futu

In [ ]:
print(shadow_dataset0[1])
print(len(shadow_dataset1))
print(len(shadow_dataset2))
print(len(shadow_dataset3))
print(len(shadow_dataset4))

(tensor([[[-2.4291, -2.2934, -2.0608,  ..., -1.1303, -1.5567, -2.0026],
         [-2.0026, -1.6924, -1.2466,  ..., -1.2466, -1.6343, -2.0220],
         [-1.4986, -1.1497, -0.7038,  ..., -1.3435, -1.6731, -2.0414],
         ...,
         [ 1.0990,  0.4593, -0.3161,  ...,  0.0328, -0.3355, -0.5875],
         [ 0.7888,  0.2848, -0.4324,  ...,  0.1104, -0.2192, -0.4518],
         [ 0.3042, -0.0641, -0.6650,  ..., -0.2192, -0.4324, -0.6069]],

        [[-2.4183, -2.2806, -2.0446,  ..., -1.0809, -1.5136, -1.9659],
         [-1.9856, -1.6709, -1.2186,  ..., -1.1989, -1.5922, -1.9856],
         [-1.4742, -1.1202, -0.6679,  ..., -1.2972, -1.6316, -2.0053],
         ...,
         [ 1.1611,  0.5121, -0.2746,  ...,  0.0991, -0.2746, -0.5302],
         [ 0.8464,  0.3351, -0.3926,  ...,  0.1778, -0.1566, -0.3926],
         [ 0.3548, -0.0189, -0.6286,  ..., -0.1566, -0.3729, -0.5499]],

        [[-2.2214, -2.0848, -1.8507,  ..., -0.8557, -1.2849, -1.7336],
         [-1.7922, -1.4800, -1.0313,  ..., -

In [ ]:
shadow_datasets = [shadow_dataset0,shadow_dataset1, shadow_dataset2, shadow_dataset3, shadow_dataset4]

In [ ]:
test_dataset = torch.load('test_dataset.pth')


test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)

<ipython-input-25-b0412d7c1c72>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  test_dataset = torch.load('test_dataset.pth')


In [ ]:
def generate_attack_data(model, data_loader, member_label):
    model.eval()
    attack_data = []
    attack_labels = []
    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            probs = model(images)
            attack_data.append(probs.cpu().numpy())
            attack_labels.append(member_label)
    return attack_data, attack_labels

In [ ]:
total_size = len(test_loader.dataset)
split_size = len(test_loader.dataset) // 5
print(split_size)
split_sizes = [split_size] * 5

# Adjust the last split to account for any remainder
if sum(split_sizes) < total_size:
    split_sizes[-1] += total_size - sum(split_sizes)


# Split the dataset into 4 parts
subset1, subset2, subset3, subset4, subset5 = random_split(test_dataset, split_sizes)

# Create DataLoaders for each subset
batch_size = 32  # Define batch size for each DataLoader
dataloader1 = DataLoader(subset1, batch_size=1, shuffle=True)
dataloader2 = DataLoader(subset2, batch_size=1, shuffle=True)
dataloader3 = DataLoader(subset3, batch_size=1, shuffle=True)
dataloader4 = DataLoader(subset4, batch_size=1, shuffle=True)
dataloader5 = DataLoader(subset5, batch_size=1, shuffle=True)

# Now you have 4 DataLoaders with approximately equal data split
test_data_loaders = [dataloader1, dataloader2, dataloader3, dataloader4,dataloader5]

2000


In [ ]:
#model architecture

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 10)
        self.tanh = nn.Tanh()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.pool(self.tanh(self.conv1(x)))
        x = self.pool(self.tanh(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)  # Flatten the tensor for the fully connected layer
        x = self.tanh(self.fc1(x))
        x = self.softmax(self.fc2(x))
        return x

In [ ]:
#load models

shadow_model0 = CNN()
shadow_model1 = CNN()
shadow_model2 = CNN()
shadow_model3 = CNN()
shadow_model4 = CNN()


In [ ]:
shadow_model0.load_state_dict(torch.load('shadow_model0.pth'))
shadow_model1.load_state_dict(torch.load('shadow_model1.pth'))
shadow_model2.load_state_dict(torch.load('shadow_model2.pth'))
shadow_model3.load_state_dict(torch.load('shadow_model3.pth'))
shadow_model4.load_state_dict(torch.load('shadow_model4.pth'))

<ipython-input-30-01cc54133076>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  shadow_model0.load_state_dict(torch.load('shadow_model0.pth'))
<ipython-input-30-01cc5413307

<All keys matched successfully>

In [ ]:
shadow_models = [shadow_model0,shadow_model1, shadow_model2, shadow_model3, shadow_model4]

In [ ]:
attack_data = []
attack_labels = []
resized_train_loaders =[]
for i in range(5):
  print(i)
  train_loader_resized = DataLoader(dataset=shadow_datasets[i], batch_size=1, shuffle=True)
  resized_train_loaders.append(train_loader_resized)
  member_data, member_labels = generate_attack_data(shadow_models[i], train_loader_resized, member_label=1)
  non_member_data, non_member_labels = generate_attack_data(shadow_models[i], test_data_loaders[i], member_label=0)
  attack_data.extend(member_data+non_member_data)
  attack_labels.extend(member_labels+non_member_labels)

0
1
2
3
4


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score

# Assuming attack_data and attack_labels are already defined as lists
# Convert attack_data to a NumPy array
attack_data = np.array(attack_data).reshape(len(attack_data), -1)


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(attack_data, attack_labels, test_size=0.2, random_state=42)

# Train a logistic regression model


print("Class distribution:", np.bincount(y_train))

Class distribution: [ 8023 39977]


In [ ]:
np.savez('train_data.npz', X=X_train, Y=y_train)

In [ ]:
from torchvision.datasets import CIFAR10

transform = transforms.Compose([
        transforms.ToTensor(),
         transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
   ])
target_train_data = CIFAR10(root='./data', train=True, transform=transform, download=True)

Files already downloaded and verified


In [ ]:
target_model = CNN()
target_model.load_state_dict(torch.load('target_model.pth'))

<ipython-input-36-e7345d752fdf>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  target_model.load_state_dict(torch.load('target_model.pth'))


<All keys matched successfully>

In [ ]:
member_loader_resized = DataLoader(dataset=target_train_data, batch_size=1, shuffle=True)
member_attack_data, member_attack_labels = generate_attack_data(target_model, member_loader_resized, member_label=1)
non_member_data0, non_member_labels0 = generate_attack_data(shadow_models[0], test_data_loaders[0], member_label=0)
non_member_data1, non_member_labels1 = generate_attack_data(shadow_models[1], test_data_loaders[1], member_label=0)
non_member_data2, non_member_labels2 = generate_attack_data(shadow_models[2], test_data_loaders[2], member_label=0)
non_member_data3, non_member_labels3 = generate_attack_data(shadow_models[3], test_data_loaders[3], member_label=0)
non_member_data4, non_member_labels4 = generate_attack_data(shadow_models[4], test_data_loaders[4], member_label=0)

attack_data = []
attack_labels = []
attack_data.extend(member_attack_data+non_member_data0+non_member_data1+non_member_data2+non_member_data3+non_member_data4)
attack_labels.extend(member_attack_labels+non_member_labels0+non_member_labels1+non_member_labels2+non_member_labels3+non_member_labels4)

attack_data = np.array(attack_data).reshape(len(attack_data), -1)
y_test = np.array(attack_labels)

np.save('attack_test.npy', attack_data)
np.save('attack_test_label.npy', y_test)